# Projeto Final | Analytics Engineering
----
**Engenharia de Dados e Garantia de Qualidade no Conjunto de Dados do Airbnb no Rio de Janeiro**

# Etapa 1:

**Aquisição de Dados e Armazenamento de Dados em PostgreSQL - Camada Bronze**

   - Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
   - Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

## Bibliotecas

In [ ]:
from sqlalchemy import create_engine, text as sql_text
import pandas as pd
import datetime

import os

### Chamada da biblioteca para usar o SQL

In [ ]:
%load_ext sql

## Camada Landing - Extração de dados

In [ ]:
## Importar modulo para extração dos dados
from modules.etl import Landing

url = [
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/listings.csv.gz',
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/calendar.csv.gz',
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/reviews.csv.gz'
]
data_path = './data/'
landing_step = Landing(landing_path=data_path)
landing_step.extract(url=url)
landing_step.transform()

## Preparação do Datalake

In [ ]:
import modules.database as db

db.criar_database()
db.criar_schemas()

## Camada Bronze - Ingestão e Análise dos Dados

In [ ]:
import pandas as pd
import modules.database as db

# Testar a conexão
try:
    print("Iniciando a estapa bronze.")

    ## Listings
    data_path = './data/'
    df_listings = pd.read_csv(data_path + 'listings.csv') # Carregando o csv listings em um Dataframe Python
    db.criar_tabela_df('bronze', 'listings', df_listings,'replace') # Criar e salvar o DataFrame na tabela "listings" dentro do esquema "bronze"

    ## Reviews
    df_reviews = pd.read_csv(data_path + 'reviews.csv') # Carregando o csv reviews em um Dataframe Python
    db.criar_tabela_df('bronze', 'reviews', df_reviews,'replace') # Criar e salvar o DataFrame na tabela "reviews" dentro do esquema "bronze"

    ## Calendar
    df_calendar = pd.read_csv(data_path + 'calendar.csv') # Carregando o csv calendar em um Dataframe Python
    db.criar_tabela_df('bronze', 'calendar', df_calendar,'replace') # Criar e salvar o DataFrame na tabela "calendar" dentro do esquema "bronze"

    ## Liberando memória
    del df_listings, df_reviews, df_calendar
    print("Etapas bronze executada com sucesso!!")
except Exception as e:
    print("Ocorreu um erro inesperado ao efetua a etapa bronze:", e)

# Etapa 2:

**Data Clean - Camada Silver**

   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
   - Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
   - Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

## Camada Silver - Limpeza de Dados e Criação de colunas

Leitura das tabelas bronze

In [2]:
import pandas as pd
from modules.database import engine_db

df_silver_listings = pd.read_sql_table(table_name='listings',schema='bronze',con=engine_db())
df_silver_reviews = pd.read_sql_table(table_name='reviews',schema='bronze',con=engine_db())
df_silver_calendar = pd.read_sql_table(table_name='calendar',schema='bronze',con=engine_db())

KeyboardInterrupt: 

Importação da biblioteca de limpeza

In [ ]:
from modules.utils import data_profiling, remove_special_caracters, process_dataframe, remove_outliers #, change_t_for_1

### Relatório com informações do perfil dos dados

In [ ]:
data_profiling(df_silver_listings, 'listings', 'data_profiling/')
# data_profiling(df_silver_reviews, 'reviews', 'data_profiling/')
# data_profiling(df_silver_calendar, 'calendar', 'data_profiling/')

### Processamento da tabela **listings**

##### Limpeza de caracteres

In [ ]:
df_silver_listings["neighborhood_overview"] = df_silver_listings["neighborhood_overview"].apply(remove_special_caracters)
df_silver_listings["host_about"] = df_silver_listings["host_about"].apply(remove_special_caracters)

##### Identificar e lidar com valores ausentes, duplicatas e outliers

In [ ]:
# colunas para considerar a eliminação de outliers
cols_listings = ['review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

df_process = process_dataframe(df_silver_listings, 'listings')
df_silver_listings = remove_outliers(df_process, cols_listings)

##### Conversão do T e F para 1 e 0

In [ ]:
cols_listings2 = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']

for col in cols_listings2:
    df_silver_listings[col] = df_silver_listings[col].apply(change_t_for_1).astype("int")

##### Transformação da coluna bathroom_text em 2 colunas

In [ ]:
df_silver_listings['bathrooms_text'].unique()

In [ ]:
# Preencha os valores nulos na coluna 'bathrooms_text' com '0 bath'
df_silver_listings['bathrooms_text'] = df_silver_listings['bathrooms_text'].fillna('0 bath')
    
# Defina o tipo de banheiro
df_silver_listings['bathroom_type'] = df_silver_listings['bathrooms_text'].str.replace(r'(\d+\.?\d*)', '', regex=True)
    
# Remova 's' do final de 'baths e traz o strip()'
df_silver_listings['bathroom_type'] = df_silver_listings['bathroom_type'].str.replace(r's?$', '', regex=True).apply(lambda x: x.strip())
    
# Extraia o número de banheiros usando expressões regulares
df_silver_listings['bathroom_quantity'] = df_silver_listings['bathrooms_text'].str.extract(r'(\d+\.?\d*)')[0].astype(float)
    
# Converta a coluna 'bathroom_quantity' para tipo numérico
df_silver_listings['bathroom_quantity'] = pd.to_numeric(df_silver_listings['bathroom_quantity'], errors='coerce').fillna(0)

In [ ]:
df_silver_listings['bathroom_type'].unique()

In [ ]:
df_silver_listings[["bathroom_type",'bathroom_quantity']]

##### Padronização dos nomes e tipos das colunas

In [ ]:
# Definir o tipo das colunas que não serão de texto
list_int_listings = ['id', 'host_id', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms']
list_datetime_listings = ['last_scraped']
list_float_listings = ['review_scores_location', 'review_scores_value', 'reviews_per_month']

In [ ]:
## Tipos das colunas

print('Integer')
print(df_silver_listings[list_int_listings].dtypes)

print('\nDatetime')
print(df_silver_listings[list_datetime_listings].dtypes)

print('\nFloat')
print(df_silver_listings[list_float_listings].dtypes)

In [ ]:
## Nessário apenas alterar o tipo das colunas 'last_scraped'
df_silver_listings['last_scraped'] = pd.to_datetime(df_silver_listings['last_scraped'], format='%Y-%m-%d')

In [ ]:
df_silver_listings[list_datetime_listings].dtypes

### Processamento da tabela **reviews**

##### Limpeza de caracteres

In [ ]:
df_silver_reviews["comments"] = df_silver_reviews["comments"].apply(remove_special_caracters)

##### Identificar e lidar com valores ausentes, duplicatas e outliers

In [ ]:
# colunas para considerar a eliminação de outliers
# cols_reviews = []

df_process = process_dataframe(df_silver_reviews, 'reviews')
# df_silver_reviews = remove_outliers(df_process, cols_reviews)

##### Padronização dos nomes e tipos das colunas

In [ ]:
# Definir o tipo das colunas que não serão de texto
list_int_reviews = ['listing_id', 'id', 'reviewer_id']
list_datetime_reviews = ['date']

In [ ]:
df_silver_reviews.dtypes

In [ ]:
## Apenas a alteração do tipo da coluna 'date' é necessária.
df_silver_reviews['date'] = pd.to_datetime(df_silver_reviews['date'], format='%Y-%m-%d')

In [ ]:
df_silver_reviews.dtypes

### Processamento da tabela **calendar**

##### Limpeza de caracteres

In [ ]:
df_silver_calendar["price"] = df_silver_calendar["price"].apply(remove_special_caracters).astype("float")

##### Identificar e lidar com valores ausentes, duplicatas e outliers

In [ ]:
# colunas para considerar a eliminação de outliers
cols_calendar = ['price']

df_process = process_dataframe(df_silver_calendar, 'calendar')
df_silver_calendar = remove_outliers(df_process, cols_calendar)

##### Conversão do T e F para 1 e 0

In [ ]:
cols_calendar2 = ['avaliable']

for col in cols_calendar2:
    df_silver_calendar[col] = df_silver_calendar[col].apply(change_t_for_1).astype("int")

##### Padronização dos nomes e tipos das colunas

In [ ]:
# Definir o tipo das colunas que não serão de texto
list_int_calendar = ['listing_id', 'minimum_nights', 'maximum_nights']
list_datetime_calendar = ['date']
list_float_calendar = ['price']

In [ ]:
df_silver_calendar.dtypes

In [ ]:
## Nessário apenas alterar o tipo das colunas 'date','minimum_nights', 'maximum_nights'
df_silver_calendar['date'] = pd.to_datetime(df_silver_calendar['date'], format='%Y-%m-%d')

In [ ]:
for column in ['minimum_nights', 'maximum_nights']:
    #transforma a coluna para o tipo inteiro
    df_silver_calendar[column] = df_silver_calendar[column].astype("int")

In [ ]:
df_silver_calendar.dtypes

# Etapa 3:

**Data Quality - Camada Silver**

   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
   - Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

### Função para verificar os casos nulos

In [ ]:
def check_missing(df):
    res_missing = df.isna().sum()
    res_missing = (res_missing/len(df))*100
    return res_missing[res_missing != 0]

In [ ]:
## Todas as colunas com dados faltantes 
check_missing(df_silver_listings).sort_values(ascending=False)

### Preenchimento dos campos nulos com valores fixos
- df_cln['notRepairedDamage'] = df_cln['notRepairedDamage'].fillna("no_info")

### Preenchimento dos campos nulos com o campo que mais se repete
- high_freq = df_cln['fuelType'].value_counts().idxmax()
- df_cln['fuelType'] = df_cln['fuelType'].fillna(high_freq)

### Preenchimento dos campos nulos com valores fixos de outra coluna
- df_cln['model'] = df_cln['model'].fillna(df_cln['vehicleType'])

### Verificação da precisão dos dados
ALGUMAS POSSÍVEIS VERIFICAÇÕES:

- Os meses precisam estar no intervalo: 1 <= meses <= 12
- Os anos precisam estar no intervalo: 1900 <= ano <= 2016
```python
df_cln.loc[(df_cln['yearOfRegistration']<1900) | (df_cln['yearOfRegistration']>2016), 'yearOfRegistration'] = 1900
```

- Os preços precisam ser maiores que 0

# Etapa 4:

**Testes de Qualidade - Camada Silver**

   - Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
   - Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.

# Etapa 5:

**Transformação de Dados com dbt - Camada Silver**

   - Utilize a ferramenta dbt para criar a camada "silver" de dados, realizando transformações e preparando os dados da camada em questão.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "silver" e automatize a execução das transformações.

# Etapa 6:

**Armazenamento de Dados em PostgreSQL - Camada Silver**

   - Armazene os dados da camada "silver" no mesmo banco de dados PostgreSQL.
   - Estabeleça conexões entre o dbt e o PostgreSQL para carregar os dados transformados da camada "silver" no banco.

# Etapa 7:

**Validação de Expectativas com Great Expectations - Camada Silver**

   - Implemente validações adicionais usando Great Expectations nas camadas de dados da camada "silver".
   - Monitore a qualidade dos dados da camada "silver" após cada transformação e ajuste os testes de acordo.

# Etapa 8:

**Transformação de Dados com dbt - Camada Gold**

   - Utilize o dbt para criar a camada "gold" de dados, aplicando agregações especializadas, como médias de preços por propriedade, por período, e outras agregações especializadas.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "gold" e automatize a execução das transformações.
   - Armazene os dados da camada "gold" no mesmo banco de dados PostgreSQL, mantendo a estrutura de dados otimizada para consultas analíticas.